# あなたの文章に合った「いらすとや」画像をレコメンド♪（応用編）

解説記事: https://qiita.com/sonoisa/items/27527c741ae93ddd6df4

In [0]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab
!apt-get -q -y install swig 
!pip install mecab-python3 pymagnitude ja-sentence-segmenter

In [0]:
!wget http://gensen.dl.itc.u-tokyo.ac.jp/soft/pytermextract-0_01.zip
!unzip pytermextract-0_01.zip
!cd pytermextract-0_01; python setup.py install

In [0]:
!wget "https://www.floydhub.com/api/v1/resources/Azw6ubf5y2WSRDPXJRwgra?content=true&download=true&rename=sonobe-datasets-fasttext_model-2" -O sonobe-datasets-fasttext_model-2.tar
!tar xvf sonobe-datasets-fasttext_model-2.tar

In [0]:
!wget "https://www.floydhub.com/api/v1/resources/n52RTWdCosGvud4gitpE5b/irasuto_items_part.json?content=true&rename=irasuto_items_partjson" -O irasuto_items_part.json

In [0]:
from pymagnitude import *
fasttext_model = Magnitude("jawiki.base_lowercase.ipadic.fasttext.ws5-neg5-epoch5.magnitude", normalized=True, ngram_oov=True, case_insensitive=True)

In [0]:
similarities = fasttext_model.most_similar(positive=['王子', '女'], negative=['男'])
similarities

In [0]:
import numpy as np
def cos_sim(v1, v2):
    v1 = v1 / np.linalg.norm(v1, axis=0, ord=2)
    v2 = v2 / np.linalg.norm(v2, axis=0, ord=2)
    return 1 - np.sum(v1 * v2)

def euclid_sim(v1, v2):
    return np.linalg.norm(v2 - v1, ord=2)

def l1_sim(v1, v2):
    return np.linalg.norm(v2 - v1, ord=1)

def sentence_similarity(v1, v2):
    '''文ベクトルの類似度を返す。小さいほど類似している。'''
    return cos_sim(v1, v2)

In [0]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    s = s.lower()
    return s

In [0]:
def normalize_text(text):
    return normalize_neologd(text)

In [0]:
import MeCab
mecab = MeCab.Tagger() # MeCab.Tagger("-d /usr/local/lib/mecab/dic/ipadic")

In [0]:
class Morph(object):
    def __init__(self, surface, pos, base, line):
        self.surface = surface
        self.pos = pos
        self.base = base
        self.line = line
    def __repr__(self):
        return str({
            "surface": self.surface,
            "pos": self.pos,
            "base": self.base
        })

def tokenize(sentence):
    sentence = normalize_text(sentence)
    mecab.parse("")
    lines = mecab.parse(sentence).split("\n")
    tokens = []
    for line in lines:
        elems = line.split("\t")
        if len(elems) < 2:
            continue
        surface = elems[0]
        if len(surface):
            feature = elems[1].split(",")
            base = surface if len(feature) < 7 or feature[6] == "*" else feature[6]
            pos = ",".join(feature[0:4])
            tokens.append(Morph(surface=surface, pos=pos, base=base, line=line))
    return tokens

In [0]:
tokenize("MeCabを用いて正規化済み文字列を形態素解析します！！")

In [0]:
import json

with open('irasuto_items_part.json', 'r') as items_file:
    items = json.load(items_file)

In [0]:
stop_pos = {
    "助詞,格助詞,一般,*",
    "助詞,格助詞,引用,*",
    "助詞,格助詞,連語,*",
    "助詞,係助詞,*,*",
    "助詞,終助詞,*,*",
    "助詞,接続助詞,*,*",
    "助詞,特殊,*,*",
    "助詞,副詞化,*,*",
    "助詞,副助詞,*,*",
    "助詞,副助詞／並立助詞／終助詞,*,*",
    "助詞,並立助詞,*,*",
    "助詞,連体化,*,*",
    "助動詞,*,*,*",
    "記号,句点,*,*",
    "記号,読点,*,*",
    "記号,空白,*,*",
    "記号,一般,*,*",
    "記号,アルファベット,*,*",
    "記号,一般,*,*",
    "記号,括弧開,*,*",
    "記号,括弧閉,*,*",
    "動詞,接尾,*,*",
    "動詞,非自立,*,*",
    "名詞,非自立,一般,*",
    "名詞,非自立,形容動詞語幹,*",
    "名詞,非自立,助動詞語幹,*",
    "名詞,非自立,副詞可能,*",
    "名詞,接尾,助動詞語幹,*",
    "名詞,接尾,人名,*",
    "接頭詞,名詞接続,*,*"
}

vocab = {}
for item in items:
    desc = item["desc"]
    title = item["title"]
    tokens = tokenize(desc)
    for token in tokens:
        key = token.base
        pos = token.pos
        is_stop = pos in stop_pos
        v = vocab.get(key, { "count": 0, "pos": pos , "stop": is_stop})
        v["count"] += 1
        vocab[key] = v

vocab_list = []
for k in vocab:
    v = vocab[k]
    if not v["stop"]:
        vocab_list.append((v["count"], k, v["pos"], v["stop"]))

In [0]:
vocab_list = sorted(vocab_list, reverse=True)
vocab_list[:10]

In [0]:
stop_word = [w[1] for w in vocab_list[:4]]
stop_word

In [0]:
import re
stop_word_regex = [ re.compile("^([!?]+|\)。)$")]

In [0]:
def get_token_vectors(sentence):
    tokens = tokenize(sentence)
    vecs = []
    for token in tokens:
        if is_stop(token):
            continue
        # query = token.surface
        query = token.base
        v = fasttext_model.query(query)
        # v = v / np.linalg.norm(v, axis=0, ord=2)
        vecs.append(v)
    return vecs

def get_sentence_vector(sentence):
    vecs = get_token_vectors(sentence)
    if len(vecs) == 0:
        return None
    else:
        # return np.array(vecs).max(axis=0)
        # return np.array(vecs).mean(axis=0)
        return np.array(vecs).sum(axis=0)
        # return np.concatenate([np.array(vecs).max(axis=0), np.array(vecs).mean(axis=0)])

def is_stop(token):
    return (token.pos in stop_pos 
            or token.base in stop_word 
            or any([r for r in stop_word_regex if r.match(token.base) is not None]))

In [0]:
get_sentence_vector("与えられた文から文の分散表現を計算します。")

In [0]:
from tqdm import tqdm
for item in tqdm(items):
    desc = item["desc"]
    desc_vec = get_sentence_vector(desc)
    item["vec"] = desc_vec

In [0]:
from IPython.display import display, HTML, clear_output
from html import escape

def search_irasuto(sentence, top_n=3):
    sentence_vector = get_sentence_vector(sentence)
    sims = []
    if sentence_vector is None:
        print("検索できない文章です。もう少し文章を長くしてみてください。")
    else:
        for item in items:
            v = item["vec"]
            if v is None:
                sims.append(1.0)
            else:
                sim = sentence_similarity(sentence_vector, v)
                sims.append(sim)
    
    count = 0
    for index in np.argsort(sims):
        if count >= top_n:
            break
        item = items[index]
        desc = escape(item["desc"])
        imgs = item["imgs"]
        if len(imgs) == 0:
            continue
        img = imgs[0]
        page = item["page"]
        sim = sims[index]
        display(HTML("<div><a href='" + page + "' target='_blank' rel='noopener noreferrer'><img src='" + img + "' width='100'>" + str(sim) + ": " + desc + "</a><div>"))
        count += 1

In [0]:
search_irasuto(sentence="暴走したAI", top_n=5)

In [0]:
# 青空文庫 北大路魯山人「だしの取り方」
# https://www.aozora.gr.jp/cards/001403/files/49986_37674.html
document_text = """
かつおぶしはどういうふうに選択し、どういうふうにして削るか。まず、かつおぶしの良否の簡単な選択法をご披露しよう。よいかつおぶしは、かつおぶしとかつおぶしとを叩き合わすと、カンカンといってまるで拍子木か、ある種の石を鳴らすみたいな音がするもの。虫の入った木のように、ポトポトと音のする湿っぽい匂いのするものは悪いかつおぶし。
本節と亀節ならば、亀節がよい。見た目に小さくとも、刺身にして美味い大きいものがやはりかつおぶしにしても美味だ。見たところ、堂々としていても、本節は大味で、値も亀節の方が安く手に入る。
次に削り方だが、まず切れ味のよい鉋を持つこと。切れ味の悪い鉋ではかつおぶしを削ることはむずかしい。赤錆になったり刃の鈍くなったもので、ゴリゴリとごつく削っていたのでは、かつおぶしがたとえ百円のものでも、五十円の値打ちすらないものになる。
どんなふうに削ったのがいいだしになるかというと、削ったかつおぶしがまるで雁皮紙のごとく薄く、ガラスのように光沢のあるものでなければならない。こういうのでないと、よいだしが出ない。削り下手なかつおぶしは、死んだだしが出る。生きたいいだしを作るには、どうしても上等のよく切れる鉋を持たねばならない。そしてだしをとる時は、グラグラッと湯のたぎるところへ、サッと入れた瞬間、充分にだしができている。それをいつまでも入れておいて、クタクタ煮るのではろくなだしは出ず、かえって味をそこなうばかりである。いわゆる二番だしというようなものにしてはいけない。
そこで、まず第一に、刃の切れる、台の平らな鉋をお持ちになることをお勧めしたい。かつおぶしを非常に薄く削るということは経済的であり、能率的でもある。
なお、わたしの案ずるところでは、百の家庭のうち九十九までがいい鉋を持っていまい。料理を講義する人でも、持っていないのだから、一般家庭によい鉋を持っている家は一応ないと考えて差し支えない。
さて鉋はいつでも切れるようにしておかなければならない。しかし、素人ではよく研げないから、大工とか仕事をするひとに研いでもらえばいい。そのほか、とぎや専門という商売もあるのだから、いつも大工の鉋のようによく切れるようにしておかなければ、料理をしようとする時にまごつくのがオチだ。
日本にはかつおぶしがたくさんあるので、そう重きをおいていないが、外国にあったら大変なことだ。外国人はかつおを知らないし、従ってかつおぶしを知らない。牛乳とか、バターとか、チーズのようなもの一本で料理をしている。しかし、これは不自由なことであって、かつおぶしのある日本人はまことに幸せである。ゆえに、かつおぶしを使って美味料理の能率をあげることを心がけるのがよい。味、栄養もいいし、よい材料を選べば、世界に類のないよいスープができる。
それなのに、かつおぶしに対する知識もなく、削り方も、削って使う方法も知らないのは、情けないことだ。その上削る道具もない――これはものの間違いで、大いに反省してもらいたいことだ。現在、鉋でかつおぶしを削っているのは料理屋のみであって、たいがいは道具もなくて我慢しているようである。その料理屋さえ最近削りかつおぶしを使用している。削り節にもいろいろあって、最上の削り節ならば、まずまずであるが、削り節は削り立てがいいので、時がたってはよろしくない。
鉋があっても、切れない場合が多いし、それを使用して削れないと思うくらいなら、日本料理をやめた方がいい。
料理にかぎらず、やるというのなら、どんなことでもやるのが当然で、やらなければ達成できない。かといって、この場合、料理屋の真似をしてガラスで削るのは危険だし、たくさん削る場合は間に合わないから、無理をしてかつおぶしを削ることになる。しかし、無理をすることは味が死ぬことになるのであるから、生きた味を出すためには、よく切れる鉋にかぎるのである。
鉋を持ってないひとがいたら、ここで一奮起して、大工の使用している鉋を購入するようお勧めしたい。大工の鉋一つ買うことは、値段からいっても高価ではないし、生涯なくなるものでもないのだから、不経済にはならない。要は研げないと頭からきめてかからずに、インチキ鉋の使用を一刻も早くやめる必要があろう。
さて昆布だしのことは、東京では一流の料理屋以外はあまり知らないようだ。これは、東京には昆布を使うという習慣が昔からなかったからだろう。昆布のだしは実に結構なものであって、魚の料理には昆布だしにかぎる。かつおぶしのだしでは魚の味が二つ重なるので、どうしても具合の悪いものが出来る。味のダブルということはくどいのである。昆布をだしに使う方法は、古来京都で考えられた。周知のごとく、京都は千年も続いた都であったから、実際上の必要に迫られて、北海道で産出される昆布を、はるかな京都という山の中で、昆布だしを取るまでに発達させたのである。
昆布のだしを取るには、まず昆布を水でぬらしただけで一、二分ほど間をおき、表面がほとびた感じが出た時、水道の水でジャーッとやらずに、トロトロと出るくらいに昆布に受けながら、指先で器用にいたわって、だましだまし表面の砂やゴミを落とし、その昆布を熱湯の中へサッと通す。それでいいのだ。これではだしが出たかどうか、心配なさるかも知れない。出たか出ないかはちょっと汁を吸ってみれば、無色透明でも、うま味が出ているのがわかる。量はどのくらい入れるかは実習すれば、すぐにわかる。このだしはなどの時はぜひなくてはならない。
こぶを湯にさっと通したきりで上げてしまうのは、なにか惜しいように考え、長くいつまでも煮るのは愚の骨頂、昆布の底の甘味が出て、決して気の利いただしはできない。京都辺では引出し昆布といって、鍋の一方から長い昆布を入れ、底をくぐらして一方から引き上げるというやり方もあるが、こういうきびしいやり方だと、どんなやかましい食通たちでも、文句のいいようがないということになっている。
"""

In [0]:
import functools

from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation

def split_sentences(text):
    split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
    concat_tail_te = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)(て)$", remove_former_matched=False)
    segmenter = make_pipeline(normalize_text, split_newline, concat_tail_te, split_punc2)
    return segmenter(text)

In [0]:
sentences = list(split_sentences(document_text.replace("\r\n", "\n")))

In [0]:
import termextract.mecab
import termextract.core

mecab_text = ""
for sentence in sentences:
    tokens = tokenize(sentence)
    if tokens:
        mecab_text += "".join([token.line + "\n" for token in tokens]) + "EOS\n"

In [0]:
import collections

term_frequency = termextract.mecab.cmp_noun_dict(mecab_text)
term_lr = termextract.core.score_lr(term_frequency, ignore_words=termextract.mecab.IGNORE_WORDS, lr_mode=1, average_rate=1)
term_importance = termextract.core.term_importance(term_frequency, term_lr)
term_collection = collections.Counter(term_importance)

In [0]:
irasuto_queries = []

QUERY_LIMIT = 5

for tokenized_term, score in term_collection.most_common()[:QUERY_LIMIT]:
    query = termextract.core.modify_agglutinative_lang(tokenized_term)
    irasuto_queries.append(query)
    print(query + " " + str(score))

In [0]:
from IPython.display import display, HTML, clear_output
from html import escape

def to_irasuto_dom(item, similarity):
    imgs = item["imgs"]
    if not imgs:
        return None
    desc = escape(item["desc"])
    page = item["page"]
    dom_source = "<div>" + "".join(["<img src='" + img + "' width='100' onclick='window.open(\"" + page + "\", \"_blank\", \"noopener,noreferrer\");" + """
        var thisImage = this.parentNode;
        thisImage.parentNode.setAttribute("style", "padding:10px 10px 10px 10px;");
        var unusedImages = [];
        for (var node of thisImage.parentNode.childNodes) {
            if (node !== thisImage) {
                unusedImages.push(node);
            } else {
                for (var siblingNode of node.childNodes) {
                    if (siblingNode !== this) {
                        unusedImages.push(siblingNode);
                    }
                }
            }
            for (var imgNode of node.getElementsByTagName("img")) {
                imgNode.removeAttribute("onclick");
                /* imgNode.setAttribute("style", Math.random() >= 0.5? "float:left" : "float:right"); */
            }
        }; 
        for (var node of unusedImages) {
            node.remove();
        }'>""" for img in imgs]) + "<span>" + str(similarity) + ": " + desc + "</span></div>"
    return dom_source

def to_irasuto_recommendation_dom(query, top_n=3):
    query_vector = get_sentence_vector(query)
    sims = []
    if query_vector is None:
        #print("検索できない文章です。もう少し文章を長くしてみてください。")
        return None
    else:
        for item in items:
            v = item["vec"]
            if v is None:
                sims.append(1.0)
            else:
                sim = sentence_similarity(query_vector, v)
                sims.append(sim)
    
    count = 0
    irasuto_doms = []
    for index in np.argsort(sims):
        if count >= top_n:
            break
        irasuto_dom = to_irasuto_dom(items[index], sims[index])
        if irasuto_dom is not None:
            irasuto_doms.append(irasuto_dom)
            count += 1
    
    return ("<div style='background:rgb(253,243,243); padding:10px 10px 10px 10px;'><h3>キーワード「" 
            + escape(query) + "」に合いそうな画像を探してみました。気に入ったものをクリックしてください。</h3>" 
            + "".join(irasuto_doms) + "<button onclick='this.parentNode.remove();'>この中にはない</button></div>")

In [0]:
queries = irasuto_queries
document_html = ""
for sentence in sentences:
    document_html += "<span>" + escape(sentence) + "</span><br>"
    for query in queries:
        if query in sentence:
            document_html += to_irasuto_recommendation_dom(query=query, top_n=10)
    queries = [query for query in queries if query not in sentence]
document_html += ""
display(HTML(document_html))

In [0]:
search_irasuto(sentence="いらすとやさんに惜しみない拍手を", top_n=1)

In [0]:
search_irasuto(sentence="つづく", top_n=1)